## Create Raw Feature Vectors from InceptionV3

Will use feature vector to compute cosine similarity to evaluate whether similar watches are grouped together.

In [2]:
import numpy as np
import pandas as pd
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import applications


In [ ]:
# dimensions of our images.
img_width, img_height = 299, 299

train_data_dir = 'f_vec_data_padded/train'
validation_data_dir = 'f_vec_data_padded/test'
nb_train_samples = 6672

file_order =os.listdir(train_data_dir+'/strap/') + \
    os.listdir(train_data_dir+'/bracelet/')
nb_train_samples = len(file_order)

nb_train_samples = nb_train_samples - (nb_train_samples % 8)

batch_size = 8

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

# build the Inception V3 network
model = applications.InceptionV3(
    include_top=True, 
    weights='imagenet', 
    input_shape = (img_width,img_width,3))

In [ ]:
model.summary()

In [ ]:
layer_name = 'avg_pool'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)

In [ ]:
generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

In [ ]:
bottleneck_features_train = intermediate_layer_model.predict_generator(
    generator = generator, steps = nb_train_samples // batch_size)

Save Feature Vectors

In [ ]:
np.save(open('inceptionv3_raw_feature_vectors.npy', 'wb'),
        bottleneck_features_train)

In [ ]:
np.shape(bottleneck_features_train)

Save Feature Vector Names

In [ ]:
raw_feature_vector_names = pd.DataFrame(generator.filenames)
raw_feature_vector_names.rename(columns={0:'file_name'},inplace=True)
raw_feature_vector_names.to_csv('inceptionv3_raw_feature_vectors_names.csv')

Save Model

In [ ]:
intermediate_layer_model.save('inceptionv3_raw_2048.h5')